<a href="https://colab.research.google.com/github/sylvia31096/Secure-and-Private-AI/blob/master/Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Creating my own private database

In [0]:
import torch

In [0]:
num_records = 5000 #number of 5000 records

In [0]:
db = torch.rand(num_records)>0.5
db

Create function to make a parallel database

In [0]:
def get_parallelDB(db,removable_index):
  
  return torch.cat((db[0:removable_index],db[removable_index+1:]))

Create a function to create the database

In [0]:
def get_parallelDBs(db):
  
  parallelDBs = []
  for i in range(len(db)):
    parallelDB = get_parallelDB(db,i)
    parallelDBs.append(parallel_column)
    
    
  return parallelDBs

In [0]:
get_parallelDBs(db)

[tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 0, 0,  ..., 1, 1